In [57]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using StaticArrays
using Rotations
using Distances
using JLD2
using GeometryBasics
using GLMakie

In [58]:
function get_overlap(input, output)
    fr = MorphoMol.Utilities.get_flat_realization(output["states"][argmin(output["Es"])], input["template_centers"])
    MorphoMol.Energies.get_geometric_measures_and_overlap_value(fr, length(input["template_radii"]), [input["template_radii"]; input["template_radii"]], 0.3665, 0.0, 1.0, 100.0)[5]
end

get_overlap (generic function with 1 method)

In [59]:
folder = "../../Data/hpc_out/op_grid/narrow/2/"

for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output

            id = parse(Int, split(file, "_")[1])
            theta = MorphoMolNotebooks.get_theta(input, output)

            #ol = get_overlap(input, output)
            # if ol <= 0.01
            #     println("$(id): $(theta) | $(ol) | $(input["persistence_weights"])")
            # end
            if theta <= 4.0
                ol = get_overlap(input, output)
                println("$(id): $(theta) | $(ol) | $(input["persistence_weights"])")
            end
        catch e
            println(e)
            println("Error in $file")
        end
    end
end

104: 3.940726703939235 | 376.57577745752144 | [1.0, -0.8, -1.0]
114: 3.8477845458539384 | 428.86578559081426 | [1.0, -0.8, -0.8]
130: 3.6492648523959077 | 438.6501299756315 | [1.0, -0.8, -0.5]
144: 3.389915852791353 | 470.03145085871375 | [1.0, -0.8, -0.2]
162: 3.9231072721242035 | 448.77088088541103 | [1.0, -0.7, -0.8]
186: 3.638369251664876 | 475.3479604200646 | [1.0, -0.7, -0.3]
195: 3.3949402830461857 | 271.22450531369105 | [1.0, -0.7, -0.2]
196: 3.9470726312989552 | 435.2551599408209 | [1.0, -0.7, -0.1]
205: 3.4835014022105897 | 376.6035349830598 | [1.0, -0.6, -1.0]
207: 3.9032446698483163 | 491.21075855963596 | [1.0, -0.6, -0.9]
210: 3.2128678975592853 | 340.6824153830628 | [1.0, -0.6, -0.9]
234: 3.673562133909744 | 352.78550805142146 | [1.0, -0.6, -0.4]
240: 3.866751073783297 | 465.26944103521043 | [1.0, -0.6, -0.3]
271: 3.837684211131309 | 512.1906793602469 | [1.0, -0.5, -0.6]
275: 2.9401665447198653 | 763.0178686680873 | [1.0, -0.5, -0.6]
299: 3.804305143405591 | 465.231459124

In [36]:
# minimal_states = Vector{Vector{Float64}}()
# ms_thetas = Vector{Float64}()
# for (root, dirs, _) in walkdir("../../Data/hpc_out/op_grid/")
#     for dir in dirs
#         for file in readdir(root*"/"*dir)
#             if split(file, ".")[end] == "jld2"
#                 @load root*"/"*dir*"/"*file input output
#                 push!(minimal_states, output["states"][argmin(output["Es"])])
#                 push!(ms_thetas, MorphoMolNotebooks.get_theta(input, output))
#             end
#         end
#     end
# end
    

In [60]:
minimal_states = Vector{Vector{Float64}}()
ms_thetas = Vector{Float64}()
root = ("../../Data/hpc_out/op_grid/narrow/2/")
for file in readdir(root)
    if split(file, ".")[end] == "jld2"
        @load root*"/"*file input output
        push!(minimal_states, output["states"][argmin(output["Es"])])
        push!(ms_thetas, MorphoMolNotebooks.get_theta(input, output))
    end
end

In [61]:
function persistence(x::Vector{Float64}, template_centers::Matrix{Float64}, persistence_weights::Vector{Float64})
    flat_realization = MorphoMol.Utilities.get_flat_realization(x, template_centers)
    points = Vector{Vector{Float64}}([e for e in eachcol(reshape(flat_realization, (3, Int(length(flat_realization) / 3))))])
    pdgm = MorphoMol.Energies.get_alpha_shape_persistence_diagram(points)
    p0 = MorphoMol.Energies.get_total_persistence(pdgm[1], persistence_weights[1])
    p1 = MorphoMol.Energies.get_total_persistence(pdgm[2], persistence_weights[2])
    p2 = MorphoMol.Energies.get_total_persistence(pdgm[3], persistence_weights[3])
    p0 + p1 + p2, Dict{String, Any}("P0s" => p0, "P1s" => p1, "P2s" => p2)
end

persistence (generic function with 1 method)

In [62]:
@load "$(folder)12_grid_search_op_2_6r7m.jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [66]:
pws = [1.0, -1.0, 0.0]
tc = input["template_centers"]
Es = [persistence(x, tc, pws)[1] for x in minimal_states];

In [65]:
ms_thetas[argmin(Es)]

9.56514824344531

In [56]:
display(MorphoMolNotebooks.visualize_molecules(MorphoMol.Utilities.get_point_vector_realization(minimal_states[argmin(Es)], tc), [input["template_radii"]; input["template_radii"]], 2))

GLMakie.Screen(...)

In [8]:
@load "$(folder)12_grid_search_op_2_6r7m.jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [9]:
n = length(output["Es"])
e_mindex = argmin(output["Es"])

figure_config = Dict(
    "plot_ms" => 5,
    "tracker_ms" => 10,
    "title_fs" => 20,
    "vis_range" =>  maximum([25, e_mindex-2500]):minimum([e_mindex + 10, n])
    #"vis_range" => 1:n
)

f = Figure(fontsize = 12)
#Slider
sl_i = Slider(f[3, 1:3], range = 1:length(output["states"][figure_config["vis_range"]]), startvalue = 1)
x = sl_i.value
cgl = GridLayout(f[1,1])
visualize_configuration_sequence!(cgl, x, input, output, figure_config)

aspgl = GridLayout(f[2, 1:2])    
visualize_alpha_shape_persistence_measures!(aspgl, x, input, output, figure_config)

etgl = GridLayout(f[2, 3])
visualize_energy_and_theta!(etgl, x, input, output, figure_config)

display(f)

GLMakie.Screen(...)